# WS_ch08D.ipynb
### WESmith 04/21/23

## COMPUTING MOLECULAR DISTANCES FROM A PDB FILE
#### (see book code in Chapter08/Distance.py)

### WS created this notebook to follow along with code from the book
### 'Bioinformatics with Python Cookbook' by Tiago Antao
#### Each recipe will have its own notebook, suffixed by A, B, etc.¶

In [ ]:
import math
import timeit
from Bio import PDB
import os

In [ ]:
data_dir  = 'data/ch08_data'

In [ ]:
repository = PDB.PDBList()
parser     = PDB.PDBParser()

In [ ]:
repository.retrieve_pdb_file('1TUP', pdir=data_dir, file_format='pdb')

In [ ]:
pdb1tup_file = os.path.join(data_dir, 'pdb1tup.ent')
p53_1tup     = parser.get_structure('P 53', pdb1tup_file)

In [ ]:
zns = []
for atom in p53_1tup.get_atoms():
    if atom.element == 'ZN':
        #print(atom, dir(atom), atom.mass, atom.element, atom.coord[0])
        zns.append(atom)
for zn in zns:
        print(zn, zn.coord)

In [ ]:
def get_closest_atoms(pdb_struct, ref_atom, distance):
    atoms = {}
    rx, ry, rz = ref_atom.coord
    for atom in pdb_struct.get_atoms():
        if atom == ref_atom:
            continue
        x, y, z = atom.coord
        my_dist = math.sqrt((x - rx)**2 + (y - ry)**2 + (z - rz)**2) 
        if my_dist < distance:
            atoms[atom] = my_dist
    return atoms

In [ ]:
for zn in zns:
    print()
    print(zn.coord)
    atoms = get_closest_atoms(p53_1tup, zn, 4)
    for atom, distance in atoms.items():
        print(atom.element, distance, atom.coord)

In [ ]:
for distance in [1, 2, 4, 8, 16, 32, 64, 128]:
    my_atoms = []
    for zn in zns:
        atoms = get_closest_atoms(p53_1tup, zn, distance)
        my_atoms.append(len(atoms))
    print(distance, my_atoms)

In [ ]:
# WS increased distance from 4.0
nexecs = 10
print(timeit.timeit('get_closest_atoms(p53_1tup, zns[0], 5.0)',
                    'from __main__ import get_closest_atoms, p53_1tup, zns',
                    number=nexecs) / nexecs * 1000)

In [ ]:
def get_closest_alternative(pdb_struct, ref_atom, distance):
    atoms = {}
    rx, ry, rz = ref_atom.coord
    for atom in pdb_struct.get_atoms():
        if atom == ref_atom:
            continue
        x, y, z = atom.coord
        if abs(x - rx) > distance or abs(y - ry) > distance or abs(z - rz) > distance:
            continue
        my_dist = math.sqrt((x - rx)**2 + (y - ry)**2 + (z - rz)**2) 
        if my_dist < distance:
            atoms[atom] = my_dist
    return atoms

In [ ]:
# increased distance from 4.0
# modified algo twice as fast
print(timeit.timeit('get_closest_alternative(p53_1tup, zns[0], 5.0)',
                    'from __main__ import get_closest_alternative, p53_1tup, zns',
                    number=nexecs) / nexecs * 1000)

In [ ]:
print('Standard')
for distance in [1, 4, 16, 64, 128]:
    print(timeit.timeit('get_closest_atoms(p53_1tup, zns[0], distance)',
                        'from __main__ import get_closest_atoms, p53_1tup, zns, distance',
                        number=nexecs) / nexecs * 1000)

In [ ]:
print('Optimized') # WS gets more expensive than standard as more atoms are calculated
for distance in [1, 4, 16, 64, 128]:
    print(timeit.timeit('get_closest_alternative(p53_1tup, zns[0], distance)',
                        'from __main__ import get_closest_alternative, p53_1tup, zns, distance',
                        number=nexecs) / nexecs * 1000)